# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Polycystic Kidney Disease'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [16]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [17]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [18]:
gender_col = 'gender'

In [19]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [23]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Thyroid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Polycystic-Kidney-Disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['.DS_Store', 'GSE74451', 'GSE74453']

Repeat the below steps for all the accession numbers

In [33]:
cohort = accession_num = "GSE74451"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Polycystic-Kidney-Disease/GSE74451/GSE74451_family.soft.gz',
 '../../DATA/GEO/Polycystic-Kidney-Disease/GSE74451/GSE74451_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [34]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Identification of a novel risk factor for intracranial aneurysms in ADPKD using iPSC models [Agilent]"
!Series_summary	"Cardiovascular complications are the leading cause of death in autosomal dominant polycystic kidney disease (ADPKD), and intracranial aneurysm (ICA) causing subarachnoid hemorrhage is among the most serious complications. The diagnostic and therapeutic strategies for ICAs in ADPKD have not been fully established. We here generated induced pluripotent stem cells (iPSCs) from seven ADPKD patients, including four with ICAs. The vascular cells differentiated from ADPKD-iPSCs showed altered Ca2+ entry and gene expression profiles compared with those from control-iPSCs. We found that the expression level of a metalloenzyme gene, matrix metalloproteinase (MMP) 1, was specifically elevated in the iPSC-derived endothelia from ADPKD patients with ICAs. Furthermore, we confirmed a statistically significant correlation between the serum MMP1 levels and the developm

In [35]:
clinical_data.head()

,!Sample_geo_accession,GSM1920920,GSM1920921,GSM1920922,GSM1920923,GSM1920924,GSM1920925,GSM1920926,GSM1920927,GSM1920928,...,GSM1920941,GSM1920942,GSM1920943,GSM1920944,GSM1920945,GSM1920946,GSM1920947,GSM1920948,GSM1920949,GSM1920950
0,!Sample_characteristics_ch1,subject/sample source id: TIG114 4F1,subject/sample source id: TIG118 4F1,subject/sample source id: TIG119 4F1,subject/sample source id: TIG121 4F4,subject/sample source id: TIG975E4,subject/sample source id: 585A1,subject/sample source id: 585B1,subject/sample source id: TIG114 4F1,subject/sample source id: TIG118 4F1,...,subject/sample source id: CiRA00008,subject/sample source id: CiRA00009,subject/sample source id: CiRA00010,subject/sample source id: CiRA00004,subject/sample source id: CiRA00005,subject/sample source id: CiRA00006,subject/sample source id: CiRA00007,subject/sample source id: CiRA00008,subject/sample source id: CiRA00009,subject/sample source id: CiRA00010
1,!Sample_characteristics_ch1,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,subject status: healthy control,...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...,subject status: autosomal dominant polycystic ...
2,!Sample_characteristics_ch1,gender: Male,gender: Female,gender: Male,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,gender: Female,...,gender: Male,gender: Female,gender: Female,gender: Female,gender: Female,gender: Male,gender: Male,gender: Male,gender: Female,gender: Female
3,!Sample_characteristics_ch1,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,...,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived endothelial cells,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...,cell type: iPSC derived vascular smooth muscle...
4,!Sample_characteristics_ch1,gender: Male,gender: Female,gender: Male,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,gender: Female,...,gender: Male,gender: Female,gender: Female,gender: Female,gender: Female,gender: Male,gender: Male,gender: Male,gender: Female,gender: Female


In [36]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['subject/sample source id: TIG114 4F1',
  'subject/sample source id: TIG118 4F1',
  'subject/sample source id: TIG119 4F1',
  'subject/sample source id: TIG121 4F4',
  'subject/sample source id: TIG975E4',
  'subject/sample source id: 585A1',
  'subject/sample source id: 585B1',
  'subject/sample source id: TIG107 4F1',
  'subject/sample source id: TIG120 4F1',
  'subject/sample source id: TIG120 3F7',
  'subject/sample source id: TIG975E2',
  'subject/sample source id: CiRA00004',
  'subject/sample source id: CiRA00005',
  'subject/sample source id: CiRA00006',
  'subject/sample source id: CiRA00007',
  'subject/sample source id: CiRA00008',
  'subject/sample source id: CiRA00009',
  'subject/sample source id: CiRA00010'],
 1: ['subject status: healthy control',
  'subject status: autosomal dominant polycystic kidney disease (ADPKD) patient'],
 2: ['gender: Male', 'gender: Female'],
 3: ['cell type: iPSC derived endothelial cells',
  'cell type: iPSC derived vascular smooth muscl

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [37]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Polycystic Kidney Disease\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Polycystic Kidney Disease\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait i

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [38]:
age_row = gender_row = None
convert_age = convert_gender = None

In [39]:
is_gene_availabe = True
trait_row = 1
age_row = None
gender_row = 2

trait_type = 'binary'

In [40]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [41]:
def convert_age(value):
    try:
        # Split the string and extract the age part
        age_str = value.split(': ')[-1]
        return int(age_str)  # Convert the extracted age to an integer
    except (ValueError, TypeError, IndexError):
        return None  # Unable to convert or unknown age
    
def convert_trait(value):
    if value == 'subject status: autosomal dominant polycystic kidney disease (ADPKD) patient':
        return 1
    elif value == 'subject status: healthy control':
        return 0
    else:
        return None

def convert_gender(value):
    """Converts the gender to binary format."""
    if value == 'gender: Male':
        return 1
    elif value == 'gender: Female':
        return 0
    else:
        return None


In [42]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1920920,GSM1920921,GSM1920922,GSM1920923,GSM1920924,GSM1920925,GSM1920926,GSM1920927,GSM1920928,GSM1920929,...,GSM1920941,GSM1920942,GSM1920943,GSM1920944,GSM1920945,GSM1920946,GSM1920947,GSM1920948,GSM1920949,GSM1920950
Polycystic Kidney Disease,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Gender,1,0,1,1,0,1,1,1,0,0,...,1,0,0,0,0,1,1,1,0,0


### Genetic data preprocessing and final filtering

In [43]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1920920,GSM1920921,GSM1920922,GSM1920923,GSM1920924,GSM1920925,GSM1920926,GSM1920927,GSM1920928,GSM1920929,...,GSM1920941,GSM1920942,GSM1920943,GSM1920944,GSM1920945,GSM1920946,GSM1920947,GSM1920948,GSM1920949,GSM1920950
ID,,,,,,,,,,,,,,,,,,,,,
(+)E1A_r60_1,9.246004,8.940038,9.247264,8.967822,9.195869,9.192290,9.216380,9.290020,9.487878,9.266375,...,9.687741,9.464143,9.507133,9.580722,9.326382,9.387042,9.453883,9.387452,9.708351,9.722596
(+)E1A_r60_3,-7.056843,-7.287641,-7.333660,-6.751512,-7.316245,-7.107780,-6.837537,-6.430428,-6.284057,-5.673605,...,-6.505308,-6.811409,-6.755661,-6.915263,-6.462014,-6.642806,-6.528124,-6.769742,-6.361740,-6.241919
(+)E1A_r60_a104,-6.721822,-7.242432,-7.172919,-7.250521,-7.100102,-6.409925,-6.409754,-5.756618,-5.892127,-6.214009,...,-5.952200,-6.731708,-6.352184,-5.746113,-6.077745,-5.580728,-5.822116,-6.791272,-5.423497,-4.715431
(+)E1A_r60_a107,-3.632830,-4.008791,-3.619517,-4.087158,-4.023131,-3.601093,-3.730406,-3.546134,-3.204269,-3.507256,...,-3.466101,-3.748397,-3.770009,-3.256609,-3.748391,-3.553973,-3.331778,-3.450692,-3.009829,-2.740745
(+)E1A_r60_a135,-0.644288,-1.007125,-0.678832,-1.234997,-0.927260,-0.718267,-0.818749,-0.641962,-0.376534,-0.776970,...,-0.458645,-0.928860,-0.843276,-0.376483,-0.729858,-0.590842,-0.443126,-0.482552,-0.123837,0.083241


In [44]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['(+)E1A_r60_1',
 '(+)E1A_r60_3',
 '(+)E1A_r60_a104',
 '(+)E1A_r60_a107',
 '(+)E1A_r60_a135',
 '(+)E1A_r60_a20',
 '(+)E1A_r60_a22',
 '(+)E1A_r60_a97',
 '(+)E1A_r60_n11',
 '(+)E1A_r60_n9',
 '3xSLv1',
 'A_19_P00315452',
 'A_19_P00315459',
 'A_19_P00315482',
 'A_19_P00315492',
 'A_19_P00315493',
 'A_19_P00315502',
 'A_19_P00315506',
 'A_19_P00315518',
 'A_19_P00315519']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [45]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97', '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452', 'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519']\n"

If not required, jump directly to the gene normalization step

In [46]:
requires_gene_mapping = True

In [47]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc|THC2478474'], 'CHROMOSOM

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [48]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_id_key = key_name1\ngene_symb_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing f

In [49]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [50]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

29 input query terms found dup hits:	[('A2MP1', 2), ('AACSP1', 2), ('ABCC13', 2), ('ABCC6P1', 2), ('ACTR3BP2', 2), ('ACTR3BP5', 3), ('ADA
64 input query terms found no hit:	['1/2-SBSRNA4', 'A2LD1', 'AARS', 'AATK-AS1', 'ABP1', 'ACCN1', 'ACCN2', 'ACCN3', 'ACCN4', 'ACCN5', 'A
20 input query terms found dup hits:	[('ARL2BPP2', 2), ('ARMCX3-AS1', 2), ('ASB9P1', 2), ('ASMTL-AS1', 3), ('ATP6AP1L', 2), ('ATP8B5P', 2
281 input query terms found no hit:	['ARMC4', 'ARNTL', 'ARNTL2', 'ARPM1', 'ARSE', 'ARVP6125', 'ASNA1', 'ATHL1', 'ATP1A1OS', 'ATP5A1', 'A
4 input query terms found dup hits:	[('C2orf83', 2), ('C3P1', 2), ('C5orf60', 2), ('CA5BP1', 2)]
570 input query terms found no hit:	['C17orf103', 'C17orf104', 'C17orf105', 'C17orf108', 'C17orf109', 'C17orf110', 'C17orf13', 'C17orf28
19 input query terms found dup hits:	[('CCDC144NL', 2), ('CCDC162P', 2), ('CD99P1', 3), ('CDR1', 2), ('CDRT15P1', 2), ('CDRT15P2', 2), ('
61 input query terms found no hit:	['CCDC129', 'CCDC130', 'CCDC132', 'CCDC135',

In [51]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [52]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 31 samples.


In [53]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Polycystic Kidney Disease', the least common label is '1.0' with 14 occurrences. This represents 45.16% of the dataset.
The distribution of the feature 'Polycystic Kidney Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 15 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [54]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [55]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE74451,True,True,False,False,True,31,
1,GSE74453,True,True,False,False,True,31,


In [11]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Gender'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE74451'

In [12]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE74451,True,True,False,False,True,31,
1,GSE74453,True,True,False,False,True,31,


In [13]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Polycystic Kidney Disease,Gender,A1BG,A1BG-AS1,A1CF,A2M,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,1.0,0.777578,-0.031899,-7.052572,2.396515,-7.083835,-7.425731,2.804982,-7.272234,...,1.470103,2.400911,-4.278163,1.304596,-0.821672,-6.820560,-0.045927,4.279770,1.738270,1.128268
1,0.0,0.0,0.019992,0.399478,-6.938511,3.753389,-7.398698,-7.517762,2.968176,-7.668136,...,0.890041,2.413140,-4.454893,1.508252,-0.785966,-6.744974,0.214816,3.811045,2.255619,1.009094
2,0.0,1.0,-0.339473,0.491183,-6.495134,2.026154,-7.245238,-7.055791,2.908653,-7.448484,...,1.461866,2.637536,-4.616670,1.322132,-0.657490,-6.710567,-0.142284,4.100192,2.016346,1.139125
3,0.0,1.0,-1.234234,-0.043165,-6.282820,2.604408,-7.348601,-7.749988,3.169038,-6.342183,...,1.076129,2.284595,-4.187583,1.438889,-1.085706,-6.749612,-0.004093,3.967128,2.234359,1.119471
4,0.0,0.0,-0.343582,0.327540,-6.263671,3.323712,-7.548939,-6.871136,6.282518,-7.604585,...,0.548717,2.103316,-4.749416,0.905256,-0.884113,-7.316437,0.054186,3.899565,1.711138,0.835255


In [14]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=[], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [15]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [16]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [17]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 56.67% ± 22.61%


In [18]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 100.00% ± 0.00%


In [19]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [20]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Gender
Coefficient: -0.0500
Found 24 genes with non-zero coefficients associated with the trait 'Polycystic Kidney Disease' conditional on the factor 'Gender'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
    DMXL2   -10.822747             10.822747
 C1orf210    10.208359             10.208359
    BIRC3    -9.337225              9.337225
   ZNF594    -7.857833              7.857833
    PVRIG    -7.329107              7.329107
     JPH1     5.983102              5.983102
LOC389765    -5.059685              5.059685
   MICAL2     4.425726              4.425726
    SAMD5    -3.244684              3.244684
    GPR82    -2.912366              2.912366
     LCTL     2.841218              2.841218
  FAM111A    -2.462449              2.462449
   VPS26B     2.370523              2.370523
  DCLRE1A    -2.359259              2.359259
     OXR1    -2.235481            

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [21]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Gender
Coefficient: -0.0500
Found 97 significant genes associated with the trait 'Polycystic Kidney Disease' conditional on the factor 'Gender', with corrected p-value < 0.05:
      Variable  Coefficient      p_value  corrected_p_value
          CHAT    -0.407131 1.646791e-09           0.000030
       PCDHGC4    -0.348677 2.682962e-08           0.000245
         SMCR2    -0.400494 4.862761e-08           0.000296
         WDR93    -0.338475 1.887502e-07           0.000862
         MED29    -0.328084 2.707427e-07           0.000989
          CRNN    -0.340412 3.321832e-07           0.001011
        HAPLN2    -0.342737 5.128960e-07           0.001339
         HMGCL    -0.322867 6.246477e-07           0.001426
      SACS-AS1    -0.347340 7.512257e-07           0.001525
        GTSCR1    -0.320569 1.246130e-06           0.002277
        LY6G6E    -0.297001 1.396792e-06           0.002320
        OR10A5    -0.349958 1.617144e-06      

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
